In [4]:
%%capture
!pip install -qU bitsandbytes transformers datasets accelerate loralib einops xformers accelerate peft

In [5]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

In [10]:
notebook_login()

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_8bit = True
)

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map = device,
    trust_remote_code = True,
    quantization_config = bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [15]:
messages = [
    {"role": "system",
     "content": "Assume you are a chess master who can provide explanation for a chess move."},
    {"role": "user",
     "content": "A Knight in chess moves like L, so if it's on g1, the legal move can be computed by - 2 moves forward so g3, then either left or right so f3 or h3. Based on that, what are legal moves if a knight is at b1?"}
]

In [17]:
prompt = '''
<s>[INST] You are a Chess Master. Your task is to provide legal moves for a chess piece based on its current location, for example -
Knight at g1[/INST]
A Knight in chess moves like L.
So if it's on g1, the legal move can be computed by - 2 moves forward so g3, then either left or right so f3 or h3.
So legal moves for Knight at g1 are g3, f3, h3.</s>
[INST]
Knight at b1
[/INST]
'''

In [18]:
# #https://huggingface.co/docs/transformers/main/chat_templating
# encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

In [20]:
encoding = tokenizer(prompt, return_tensors="pt").to(device)

In [22]:
generated_ids = model.generate(input_ids = encoding.input_ids, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
<s> [INST] You are a Chess Master. Your task is to provide legal moves for a chess piece based on its current location, for example - 
Knight at g1[/INST]
A Knight in chess moves like L.
So if it's on g1, the legal move can be computed by - 2 moves forward so g3, then either left or right so f3 or h3.
So legal moves for Knight at g1 are g3, f3, h3.</s> 
[INST]
Knight at b1
[/INST]
A Knight in chess moves like L-shape. So if it's on b1, the legal move can be computed by moving 2 squares in a horizontal direction and then 1 square perpendicular to that.
So legal moves for Knight at b1 are c3 and d4.
Alternatively, you can also calculate it by moving 2 squares vertically and then 1 square to the right or left, so e2 and c2 are also legal moves.</s>


In [25]:
prompt = '''
[INST]
Bishop at c1
[/INST]
'''

In [26]:
encoding = tokenizer(prompt, return_tensors="pt").to(device)
generated_ids = model.generate(input_ids = encoding.input_ids, attention_mask = encoding.attention_mask, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
[INST]
Bishop at c1
[/INST]
In the game of chess, having a Bishop placed on c1 (the square at the bottom-left corner of the board for white, or top-right corner for black) means that the Bishop can move in a diagonal pattern, both to the left or to the right and forwards or backwards. Its initial moves could be to d2 or a2, depending on the color of the pieces. A Bishop is generally considered a powerful and flexible piece since it can control many squares along its diagonal path. However, it can't move horizontally or vertically. A Bishop can form various useful patterns, such as a 'Bishop pair' when there are two Bishops of the same color on the board, enhancing their control over the center and diagonals.

Here's a simple example of a Bishop's moves starting from the c1 square:
1. e4 (white advances their pawn)
2. Bc1 (white moves the Bishop to c1)
3. d3 (white advances their d-pawn, preventing black from occupying the d5 square)
4. Be2 (white's Bishop moves to e2 to control th